# Gausian Mixture Model for Bird Data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm


In [ ]:
# data = pd.read_csv('ebd_US-CO_202001_202312_smp_relMar-2024/ebd_US-CO_202001_202312_smp_relMar-2024.txt', sep = '\t')
# data = data[['COMMON NAME', 'COUNTY', 'COUNTY CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE', 'NUMBER OBSERVERS']]
# data.to_csv('bird_data.csv')

data = pd.read_csv('bird_data.csv')

data.head(5)

df = data[['COMMON NAME', 'LATITUDE', 'LONGITUDE']]

In [ ]:
plt.scatter(df['LATITUDE'], df['LONGITUDE'])
plt.title('Initial Scatter Plot')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.show()

In [ ]:
# Used the starter code on canvas

def GMM(dat, k):
    ##
    
    # define some variables
    covars = np.zeros((k,2,2)) # covariance matrix
    means = np.zeros((k,2)) # mean values of each component
    p_class = np.zeros(k) # W_m (weights/likelihood of each component)
    p_data_given_class = np.zeros((len(dat),k)) # P(x_i | cluster = m)
    
    # initialize means, covs, p_classes
    init_idx = np.random.choice(range(len(dat)), size=k, replace=False) # initalizations of starting points used to initialize means
    
    # initialize covariance matrix, means, and p_class for each k
    for dim in range(k):
        # set cov matrix of each component to the cov of the entire dataset
        covars[dim,:,:] = np.cov(np.transpose(dat))
        
        # set initial means to the initial chosen data points
        means[dim, :] = dat.iloc[init_idx[dim]]
        
        # give each component equal weighting to start
        p_class[dim] = 1/k
        
    # iterate: for loop alternating between: (1) Expectation Step and (2) Maximization Step
    for _ in range(50):
        
        # Expectation Step
        for i in range(k):
            p_data_given_class[:,i] = stats.multivariate_normal.pdf(dat, mean=means[i], cov=covars[i])
        
        p_class_given_data = p_data_given_class * p_class
        p_class_given_data = p_class_given_data / p_class_given_data.sum(axis=1,keepdims=True)
        
        # Maximization Step
        for i in range(k):
            
            # update weights
            p_class[i] = np.mean(p_class_given_data[:,i])
            
            # update means
            means[i] = np.average(dat, weights=p_class_given_data[:,i],axis=0)
            
            # update covariances
            diff = dat - means[i]
            covars[i] = np.dot(p_class_given_data[:,i]*diff.T,diff) / np.sum(p_class_given_data[:,i])
        
    # compute mean distance
    dat_array = dat.to_numpy()
    mean_dist = np.sum(np.sum(p_class_given_data * np.linalg.norm(dat_array[:, np.newaxis, :] - means, axis=2), axis=1)) / dat_array.shape[0]

    return p_class_given_data, means, covars, p_class, mean_dist

In [ ]:
k_values = [1,2,3,4,5,6,7,8]
distances = []

for k in k_values:
    p_class_given_data, means, covars, p_class, mean_dist = GMM(df,k)
    distances.append(mean_dist)
    
plt.plot(k_values, distances, marker = 'o')
plt.title('Elbow Plot')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Weighted-Mean-Distance to Component')
plt.show()

In [ ]:
# perform GMM for k=3
p_class_given_data, means, covars, p_class, mean_dist = GMM(df, 3)

# determine the cluster assignment for each data point
cluster_assignment = np.argmax(p_class_given_data, axis=1)

# plot data points with color according to the cluster assignment
plt.scatter(df['lat'], df['lng'], c=cluster_assignment, cmap='viridis', s=10)
plt.title('k=3')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.colorbar(label='Cluster')
plt.show()
